In [ ]:
import pandas as pd
import joblib
import numpy as np
from flask import Flask, request, jsonify

In [ ]:
app = Flask(__name__)

# 1. Cargar el modelo entrenado al iniciar la app
pipeline = joblib.load('modelo_churn.pkl')
print("Modelo cargado y listo.")

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Recibir JSON del Backend Java
        data = request.json

        # Convertir a DataFrame (1 sola fila)
        input_df = pd.DataFrame([data])

        # --- PASO 1: Predecir ---
        prediction = pipeline.predict(input_df)[0] # 0 o 1
        probability = pipeline.predict_proba(input_df)[0][1] # Ej: 0.85

        # --- PASO 2: Calcular la EXPLICABILIDAD (El "Por qué") ---
        # Accedemos a las partes internas del pipeline
        model = pipeline.named_steps['classifier']
        preprocessor = pipeline.named_steps['preprocessor']

        # Transformamos los datos del cliente igual que en el entrenamiento
        input_transformed = preprocessor.transform(input_df)

        # Obtenemos los nombres de las variables transformadas
        # (Esto es necesario porque "Region" se convierte en "Region_Asia", "Region_Europe", etc.)
        feature_names = (preprocessor.named_transformers_['num'].get_feature_names_out().tolist() +
                         preprocessor.named_transformers_['cat'].get_feature_names_out().tolist())

        # Obtenemos los coeficientes del modelo (qué peso tiene cada variable)
        coefficients = model.coef_[0]

        # Calculamos la CONTRIBUCIÓN: Valor * Peso
        # Si input_transformed es una matriz dispersa, la convertimos a array
        if hasattr(input_transformed, "toarray"):
            input_transformed = input_transformed.toarray()

        contributions = input_transformed[0] * coefficients

        # Encontramos cuál variable tuvo el valor más alto
        max_index = np.argmax(contributions)
        main_factor_name = feature_names[max_index]

        # --- PASO 3: Construir Respuesta ---
        response = {
            "prediction": "Va a Cancelar" if prediction == 1 else "Se Queda",
            "probability": round(probability, 2), # Ej: 0.85
            "main_factor": main_factor_name,      # Ej: "monthly_fee" o "region_Europe"
            #"alert_level": "RED" if probability > 0.7 else "GREEN"
        }

        return jsonify(response)

    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == '__main__':
    app.run(port=5000, debug=True)